# Data Exploration Notebook

In [1]:
import pandas as pd
# pd.set_option('max_colwidth', None)

## Train Data

In [2]:
df = pd.read_parquet('../data/train.parquet')
# df = pd.read_parquet('../data/test.parquet')

In [26]:
# Number of samples
# Distinct patent applications
# Distinct cited documents
# Distinct claim texts
# Distinct cited paragraph
# Median claim length (chars)
# Median paragraph length (chars)

df.nunique()


Unnamed: 0               2970
index                    2970
claim_id                 2926
patent_application_id    2406
cited_document_id        2445
text                     2911
text_b                   2957
label                       2
date                      248
DIznQ_0                  2970
dtype: int64

In [44]:
# df[df['text'] == df['text'].]
df[df.duplicated('cited_document_id', keep=False)].groupby('cited_document_id').count()

,Unnamed: 0,index,claim_id,patent_application_id,text,text_b,label,date,DIznQ_0
cited_document_id,,,,,,,,,
EP1003164,4,4,4,4,4,4,4,4,4
EP1041357,4,4,4,4,4,4,4,4,4
EP1083660,2,2,2,2,2,2,2,2,2
EP1092453,2,2,2,2,2,2,2,2,2
EP1126681,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...
EP928976,2,2,2,2,2,2,2,2,2
EP940686,2,2,2,2,2,2,2,2,2
EP959524,4,3,4,4,4,4,4,4,4


In [45]:
df[df['cited_document_id'] == 'EP1003164']

,Unnamed: 0,index,claim_id,patent_application_id,cited_document_id,text,text_b,label,date,DIznQ_0
261,548212.0,548212.0,36373_1,EP2575004A1,EP1003164,The device of any one of claim 2 to 5 characte...,Next Fig.13 is a PAD diagram showing the proce...,1.0,20130403.0,0.730541
1044,547999.0,547999.0,36368_1,EP2575004A1,EP1003164,An electronic device characterized by comprisi...,Next Fig.9 is a PAD diagram showing a concrete...,1.0,20130403.0,0.224242
1731,548281.0,548281.0,36375_1,EP2575004A1,EP1003164,The device of claim 5 characterized in that th...,When the initialization of the microprocessor ...,1.0,20130403.0,0.125344
1933,548281.0,548281.0,36375_1,EP2575004A1,EP1003164,The device of claim 5 characterized in that th...,When the initialization of the microprocessor ...,1.0,20130403.0,0.125344
